In [1]:
import numpy as np
import pandas as pd
import torch
import csv
import transformers
import torch.nn as nn
import torch.utils.data as Data
import torch.nn.functional as F
import json
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModel, BertConfig, get_linear_schedule_with_warmup, BertForSequenceClassification


In [2]:
import torch
print(torch.cuda.is_available())

True


In [3]:
device = 'cuda'

In [4]:
from bert_gru.model import BERT_GRU
from bert_lstm.model import BERT_LSTM

config = BertConfig.from_pretrained('bert-base-chinese')
config.num_labels = 3
m1 = BertForSequenceClassification(config = config)
m1.load_state_dict(torch.load('bert_linear/bert_cla.ckpt'))
m1.to(device)

config = BertConfig.from_pretrained('hfl/chinese-bert-wwm')
config.num_labels = 3
m2 = BertForSequenceClassification(config = config)
m2.load_state_dict(torch.load('hfl_bert_linear/bert_cla.ckpt'))
m2.to(device)

config = BertConfig.from_pretrained('hfl/chinese-roberta-wwm-ext')
config.num_labels = 3
m3 = BertForSequenceClassification(config = config)
m3.load_state_dict(torch.load('RoBERTa_linear/bert_cla.ckpt'))
m3.to(device)

m4 = BERT_GRU()
m4.load_state_dict(torch.load('bert_gru/bert_cla.ckpt'))
m4.to(device)

m5 = BERT_LSTM()
m5.load_state_dict(torch.load('bert_lstm/bert_cla.ckpt'))
m5.to(device)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias'

BERT_LSTM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  

创建dataset类

In [5]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese')

创建load_dataset function

In [6]:
from transformers import AutoTokenizer
import csv
def load_dataset(filepath, max_len, tokenizer = None):
    if tokenizer == None:
        tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese')
    
    label = []
    sentences = []
    # load dataset
    f = open(filepath, 'r', encoding='utf-8')
    r = csv.reader(f)
    for item in r:
        if r.line_num == 1:
            continue
        label.append(int(item[0]))
        sentences.append(item[1])
        
    input_ids = []
    attention_masks = []

    
    # For every sentence...
    for data in sentences:
        encoded_data = tokenizer.encode_plus(
            text=data,                      # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=max_len,             # Max length to truncate/pad
            padding='max_length',           # Pad sentence to max length
            return_attention_mask=True,      # Return attention mask
            truncation= True
            )
        
        # Add the outputs to the lists
        input_ids.append(encoded_data.get('input_ids'))
        attention_masks.append(encoded_data.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)
    labels = torch.tensor(label)
    return input_ids, attention_masks, labels


load dataset

In [7]:
import os
path = os.path.abspath(os.path.dirname(os.getcwd()))

In [8]:
os.getcwd()

'f:\\2022 IP\\实战\\_final'

In [9]:
test_dataset = load_dataset(f'{os.getcwd()}\\data\\test.csv', max_len = 32)

In [10]:
from torch.utils.data import TensorDataset

test_data = TensorDataset(test_dataset[0],test_dataset[1],test_dataset[2])

In [11]:
print('开始测试...')
m1.eval()
m2.eval()
m3.eval()
m4.eval()
m5.eval()
model = [m1, m2, m3, m4, m5]

test_result = []
for data in test_data:
    b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in data)
    b_input = b_input_ids.unsqueeze(0)  
    b_attn_mask = b_attn_mask.unsqueeze(0)
    res = np.array([0,0,0])
    for m in model:
        with torch.no_grad():
            outputs = m(b_input,b_attn_mask)
            try:
                outputs = outputs.logits
            except:
                pass
            pre = outputs.argmax()
            res[pre.item()]+=1
    r = res.argmax()
    test_result.append([b_labels.item(), r, tokenizer.convert_ids_to_tokens(b_input_ids)])

# 写入csv文件
df = pd.DataFrame(test_result)
df.to_csv('test_result.csv',index=False, header=['id', 'label','text'])

开始测试...


In [12]:
outputs.argmax()

tensor(0, device='cuda:0')

In [15]:
import pandas as pd
df = pd.read_csv('test_result.csv')
df[df.id!=df.label]

,id,label,text
6,0,1,"['[CLS]', '乱', '世', '佳', '人', '[SEP]', '[PAD]'..."
22,0,2,"['[CLS]', '刘', '军', '不', '服', '[SEP]', '[PAD]'..."
23,2,0,"['[CLS]', '打', '屎', '棍', '[SEP]', '[PAD]', '[P..."
27,0,1,"['[CLS]', '流', '氓', '兔', '哈', '哈', '[SEP]', '[..."
30,0,1,"['[CLS]', '我', '要', '尊', '严', '[SEP]', '[PAD]'..."
...,...,...,...
6516,0,1,"['[CLS]', '666', '，', '[SEP]', '[PAD]', '[PAD]..."
6522,0,2,"['[CLS]', '78', '##9', '玩', '不', '起', '[SEP]',..."
6524,1,0,"['[CLS]', '应', '该', '叫', '先', '知', '[SEP]', '[..."
6525,2,0,"['[CLS]', '3', '输', '了', '。', '[SEP]', '[PAD]'..."


In [16]:
len(df[df.id==df.label])/len(df)

0.8595256312165264